In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
data = pd.read_csv("..//..//dataset//oakland-crime-statistics-2011-to-2016//records-for-2012.csv")
print(data)

       Agency              Create Time  Area Id  Beat  Priority  \
0          OP  2012-01-01T00:00:25.000      2.0   32Y       2.0   
1          OP  2012-01-01T00:00:27.000      2.0   30Y       2.0   
2          OP  2012-01-01T00:00:48.000      1.0   06X       2.0   
3          OP  2012-01-01T00:00:58.000      2.0   35X       2.0   
4          OP  2012-01-01T00:01:14.000      1.0   02Y       2.0   
5          OP  2012-01-01T00:01:39.000      2.0   31Y       2.0   
6          OP  2012-01-01T00:01:47.000      2.0   32X       2.0   
7          OP  2012-01-01T00:02:09.000      1.0   11X       1.0   
8          OP  2012-01-01T00:02:40.000      2.0   35X       2.0   
9          OP  2012-01-01T00:02:59.000      2.0   26Y       2.0   
10         OP  2012-01-01T00:03:11.000      1.0   11X       2.0   
11         OP  2012-01-01T00:03:22.000      1.0   06X       2.0   
12         OP  2012-01-01T00:03:49.000      2.0   30Y       2.0   
13         OP  2012-01-01T00:04:11.000      1.0   06X       2.

#### 1. 删除空值处理

In [3]:
#删除所有属性中有空值的行
data1 = data.dropna()
print(data1)

       Agency              Create Time  Area Id  Beat  Priority  \
187256     OP  2012-01-11T13:17:59.000      2.0   28X       2.0   
187257     OP  2012-01-01T07:27:25.000      2.0   23X       1.0   
187258     OP  2012-01-05T07:21:49.000      1.0   12X       2.0   
187260     OP  2012-01-14T18:34:28.000      2.0   33X       2.0   
187261     OP  2012-02-15T05:20:20.000      1.0   06X       2.0   
187262     OP  2012-02-15T04:38:47.000      1.0   05X       1.0   
187263     OP  2012-03-09T11:10:03.000      1.0   10Y       2.0   
187267     OP  2012-04-10T18:17:02.000      2.0   28X       2.0   
187269     OP  2012-04-12T00:39:16.000      1.0   12X       2.0   
187270     OP  2012-04-13T14:41:16.000      1.0   25X       1.0   
187275     OP  2012-05-02T10:30:45.000      1.0   10X       2.0   
187279     OP  2012-05-15T14:42:52.000      2.0   33X       2.0   
187281     OP  2012-05-18T15:24:29.000      2.0   26Y       2.0   
187282     OP  2012-05-22T19:47:17.000      2.0   34X       2.

#### 2. 用最高频率值进行填充

In [4]:
data2 = copy.deepcopy(data)#拷贝原数据集用于填充
for i in range(10):
    count = data2.iloc[:,i].value_counts()  #依次获取每一列的series
    name = pd.DataFrame(data2.iloc[:,i]).columns.values.tolist()
    sort_ans = count.sort_values(ascending = False) #对各类别的频数值进行排序 并选择最高频率的索引
    fre = sort_ans.index[0]
    fre = str(fre)
    print(name[0]+"的最高频率值为"+fre+'\n')
    
    data2[name[0]] = data2[name[0]].fillna(fre) #进行缺失值的填充
print("------------------------------------------"+'\n'+"填充完毕")

Agency的最高频率值为OP

Create Time的最高频率值为2012-06-26T00:00:00.000

Area Id的最高频率值为1.0

Beat的最高频率值为04X

Priority的最高频率值为2.0

Incident Type Id的最高频率值为933R

Incident Type Description的最高频率值为ALARM-RINGER

Event Number的最高频率值为LOP121219000888

Closed Time的最高频率值为2012-11-19T16:18:43.000

Location 1的最高频率值为{'human_address': '{"address": "INTERNATIONAL BLVD", "city": "", "state": "", "zip": ""}'}

------------------------------------------
填充完毕


In [5]:
print(data2)  #填充结果

       Agency              Create Time Area Id  Beat Priority  \
0          OP  2012-01-01T00:00:25.000       2   32Y        2   
1          OP  2012-01-01T00:00:27.000       2   30Y        2   
2          OP  2012-01-01T00:00:48.000       1   06X        2   
3          OP  2012-01-01T00:00:58.000       2   35X        2   
4          OP  2012-01-01T00:01:14.000       1   02Y        2   
5          OP  2012-01-01T00:01:39.000       2   31Y        2   
6          OP  2012-01-01T00:01:47.000       2   32X        2   
7          OP  2012-01-01T00:02:09.000       1   11X        1   
8          OP  2012-01-01T00:02:40.000       2   35X        2   
9          OP  2012-01-01T00:02:59.000       2   26Y        2   
10         OP  2012-01-01T00:03:11.000       1   11X        2   
11         OP  2012-01-01T00:03:22.000       1   06X        2   
12         OP  2012-01-01T00:03:49.000       2   30Y        2   
13         OP  2012-01-01T00:04:11.000       1   06X        2   
14         OP  2012-01-01

#### 3. 根据数据对象的相似性来填充数据值

   这里我们使用的是KNN算法，实体之间的距离计算使用曼哈顿距离。K的值设置为6。我们选取与存在空缺值的向量距离最近的6个向量在对应属性上的平均值来对空缺值进行填充。